In [ ]:
#   This is filename:  Uhlig_impulse_resp_all.ipynb
#   It computes for impulse responses to 1% 
#   innovations for all exogenous stochastic 
#   processes in the model.

#  The inputs into this notebook are the model
#  and its solution.  See  uhlig_solver_abreviated.ipynb

#  The output of this notebook are the mactices
#  "Resp_mat"   and   "Resp_mat_named"
#  which contain impulse response values 



import numpy as np
from numpy import hstack, vstack, zeros, dot, eye, kron

import scipy as sp
from scipy import linalg as la
from scipy.linalg import null_space
import scipy.optimize as opt

import matplotlib.pyplot as plt
import pylab

np.set_printoptions(precision=4, suppress=True)


##  Choose number of periods over which 
##  impulse responses are to be computed 

Horizon = 12
#print('Horizon = ', Horizon)


m_states = QQ.shape[0]
k_exog = QQ.shape[1]
n_endog = SS.shape[0]
#print('m_states = ', m_states)
#print('n_endog = ', n_endog)
#print('k_exog = ', k_exog)



Period_list = np.matrix(range(0,Horizon+1)).reshape(1, Horizon+1)

shock_counter = 0
time_counter = 0


Resp_mat = Period_list

for shock_counter in range(k_exog):

    Response =  np.zeros((m_states+n_endog+k_exog)*Horizon).reshape(m_states+n_endog+k_exog, Horizon)
    Response = np.array(Response)
    Response[m_states+n_endog+shock_counter,0] = 1
    #print('shock_counter = ', shock_counter)
    #print('Response = ', Response)
                    
    II_Lag_01 = np.hstack(((PP, np.zeros(m_states*n_endog).reshape(m_states, n_endog), np.zeros(m_states*k_exog).reshape(m_states, k_exog))))
    II_Lag_02 = np.hstack(((RR, np.zeros(n_endog*n_endog).reshape(n_endog, n_endog), np.zeros(n_endog*k_exog).reshape(n_endog, k_exog))))
    II_Lag_03 = np.hstack(((np.zeros(k_exog*(m_states+n_endog)).reshape(k_exog, (m_states+n_endog)), NN)))
    II_Lag = np.vstack((II_Lag_01, II_Lag_02, II_Lag_03))
    #print('II_Lag = ', II_Lag)
                    
    II_Contemp_01 = np.eye(m_states+n_endog+k_exog)
    II_Contemp_02 = np.hstack((np.zeros(m_states*(m_states+n_endog)).reshape(m_states, (m_states+n_endog)), QQ))
    II_Contemp_03 = np.hstack((np.zeros(n_endog*(m_states+n_endog)).reshape(n_endog, (m_states+n_endog)), SS))
    II_Contemp_04 = np.hstack((np.zeros(k_exog*(m_states+n_endog)).reshape(k_exog, (m_states+n_endog)), np.zeros(k_exog*k_exog).reshape(k_exog, k_exog)))
    II_Contemp = II_Contemp_01+ np.vstack((II_Contemp_02, II_Contemp_03, II_Contemp_04))
    #print('II_Contemp = ', II_Contemp)
                    
    Response_0 = np.array(Response[:,0]).reshape((m_states+n_endog+k_exog),1)
    #print('Response_0 = ', Response_0)
    #print('Response_0 shape = ', Response_0.shape)
    
    Response_1 = dot(II_Contemp, Response_0).reshape((m_states+n_endog+k_exog),1)
    #print('Response_1 = ', Response_1)
    #print('Response_1 shape = ', Response_1.shape)
    
    Response = Response_1
                    
    for time_counter in range(0, Horizon):
        # print('time_counter = ', time_counter)
        Resp_col = dot(dot(II_Contemp, II_Lag), Response[:,time_counter -1])
        Response = np.hstack((Response, Resp_col))
        # print('time_counter = ', time_counter)
        # print(Response)
        
    Resp_mat = np.vstack((Resp_mat, Response, Period_list))

print('   ')
print('Resp_mat = ',Resp_mat)

    

VN_temp= np.matrix(VARNAMES)
VARNAMES = VN_temp.reshape((m_states+k_exog+n_endog),1)
VAR_NAMES = np.vstack((['Period'],VARNAMES))

for shock_counter in range(1,k_exog):    
    VAR_NAMES = np.vstack((VAR_NAMES,['Period'],VARNAMES))

VAR_NAMES =  np.vstack((VAR_NAMES,['Period']))

Resp_mat_named = np.hstack((VAR_NAMES, Resp_mat))     
print('   ')
print('Resp_mat_named = ', Resp_mat_named)



print('  ')    
print('Done. You have the matrices "Resp_mat" and "Resp_mat_named" ')
print('which contains impulse reponses to 1% innovations')
print('for all exogenous stochastic processes in the model.')
print('If you want, you can download these into an excel file')
print('or use them to make plots.')
print('  ')   
